<a href="https://colab.research.google.com/github/andrzej-malina/data_workshop_matrix/blob/master/Matrix%20III/Matrix_III_Day_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

import os
import datetime

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score
%load_ext tensorboard

In [0]:
from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

In [3]:
cd '/content/drive/My Drive/Colab Notebooks/data_workshop_matrix/Matrix III/'

/content/drive/My Drive/Colab Notebooks/data_workshop_matrix/Matrix III


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1 : y_train = to_categorical(y_train)
if y_test.ndim == 1 : y_test = to_categorical(y_test)

In [7]:
y_train.shape, y_test.shape, X_train.shape, X_test.shape

((34799, 43), (4410, 43), (34799, 32, 32, 3), (4410, 32, 32, 3))

In [0]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  
  logdir = os.path.join('logs', datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      X_train, 
      y_train,
      batch_size = params_fit.get('batch_size', 128),
      epochs = params_fit.get('epochs', 5),
      verbose = params_fit.get('verbose', 1),
      validation_data = params_fit.get('validation_data', (X_train, y_train)),
      callbacks = [tensorboard_callback]
  )

  return model

In [0]:
def predict(model_trained, X_test, y_test, scoring=accuracy_score):

  y_test_norm = np.argmax(y_test, axis=1)
  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)
  return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
      Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
      Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
      MaxPool2D(),
      Dropout(0.3),
      
      Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
      Conv2D(filters=32, kernel_size=(3,3), activation='relu'),
      MaxPool2D(),
      Dropout(0.3),
      
      Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
      Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
      MaxPool2D(),
      Dropout(0.3),

      Flatten(),

      Dense(1024, activation='relu'),
      Dropout(0.3),

      Dense(1024, activation='relu'),
      Dropout(0.3),

      Dense(num_classes, activation='softmax')
    ])

In [13]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train, params_fit={})
predict(model_trained, X_test, y_test )

Epoch 1/5
272/272 [==============================] - 6s 22ms/step - loss: 2.7121 - accuracy: 0.2512 - val_loss: 1.3254 - val_accuracy: 0.5794
Epoch 2/5
272/272 [==============================] - 5s 20ms/step - loss: 1.0790 - accuracy: 0.6525 - val_loss: 0.2848 - val_accuracy: 0.9087
Epoch 3/5
272/272 [==============================] - 5s 20ms/step - loss: 0.5036 - accuracy: 0.8395 - val_loss: 0.1030 - val_accuracy: 0.9691
Epoch 4/5
272/272 [==============================] - 5s 20ms/step - loss: 0.3368 - accuracy: 0.8967 - val_loss: 0.0903 - val_accuracy: 0.9722
Epoch 5/5
272/272 [==============================] - 5s 20ms/step - loss: 0.2555 - accuracy: 0.9221 - val_loss: 0.0525 - val_accuracy: 0.9846


0.9607709750566893

In [0]:
def get_model(params):
  return Sequential([
      Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
      Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
      MaxPool2D(),
      Dropout(params['dropout_cnn_block_one']),
      
      Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
      Conv2D(filters=32, kernel_size=(3,3), activation='relu'),
      MaxPool2D(),
      Dropout(params['dropout_cnn_block_two']),
      
      Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
      Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
      MaxPool2D(),
      Dropout(params['dropout_cnn_block_three']),

      Flatten(),

      Dense(1024, activation='relu'),
      Dropout(params['dropout_dense_block_one']),

      Dense(1024, activation='relu'),
      Dropout(params['dropout_dense_block_two']),

      Dense(num_classes, activation='softmax')
    ])

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  
  model.fit(
      X_train, 
      y_train,
      batch_size = int(params.get('batch_size', 128)),
      epochs = 5,
      verbose = 0
  )
  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print (params, 'accuracy = {}'.format(accuracy))
  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model }

In [0]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),
    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7),
}

In [37]:
best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 160.0, 'dropout_cnn_block_one': 0.49692014029357523, 'dropout_cnn_block_three': 0.32889991847051764, 'dropout_cnn_block_two': 0.4831398445141789, 'dropout_dense_block_one': 0.4706999607944915, 'dropout_dense_block_two': 0.6683504937749825}
accuracy = 0.45895692706108093
{'batch_size': 180.0, 'dropout_cnn_block_one': 0.4514442315660754, 'dropout_cnn_block_three': 0.4972654818025203, 'dropout_cnn_block_two': 0.3757113036608398, 'dropout_dense_block_one': 0.6673459060384053, 'dropout_dense_block_two': 0.6610679724757427}
accuracy = 0.5469387769699097
{'batch_size': 160.0, 'dropout_cnn_block_one': 0.44929045816899416, 'dropout_cnn_block_three': 0.3062285988786869, 'dropout_cnn_block_two': 0.46674153365460097, 'dropout_dense_block_one': 0.6120746826831973, 'dropout_dense_block_two': 0.46339062078254095}
accuracy = 0.8854875564575195
{'batch_size': 110.0, 'dropout_cnn_block_one': 0.36589857861955194, 'dropout_cnn_block_three': 0.3006289123626733, 'dropout_cnn_block_two': 0.491